In [1]:
import pandas as pd
import itertools
from gurobipy import *

In [2]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
production = pd.read_excel("base_case_data.xlsx", header = 0, index_col=0, sheet_name = "production")

setup = pd.read_excel("base_case_data.xlsx", header = 0, index_col=0, sheet_name = "setup")

demand = pd.read_excel("base_case_data.xlsx", header = 0, index_col=0, sheet_name = "demand")

In [3]:
# set up decision variables
# reference: https://stackoverflow.com/questions/1953194/permutations-of-two-lists-in-python
part_names = list(setup.columns)
machine_names = list(setup.index)

# this is the working time in a week
var_list = []
for r in itertools.product(machine_names, part_names):
    var_list.append(r[0]+ "_" +r[1]) 
# var_list

In [4]:
# cost contribution
regular_cost = {}
for v in var_list:
    first = v.split("_")[0]
    second = v.split("_")[1]
    regular_cost[v] = 100

overtime_cost = {}
for v in var_list:
    first = v.split("_")[0]
    second = v.split("_")[1]
    regular_cost[v] = 150

In [5]:
# create dictionary for objective coefficient
setup_co = {}
for v in var_list:
    first = v.split("_")[0]
    second = v.split("_")[1]
    setup_co[v] = setup.loc[first, second]

In [6]:
# maximum regular working hours per week
regular_hour_right_leq = {}
for name in machine_names:
    regular_hour_right_leq[name] = 120
# constraint_right_eq

# maximum overtime hours per week
overtime_hour_right_leq = {}
for name in machine_names:
    overtime_hour_right_leq[name] = 48

In [7]:
# Demand of each part
constraint_left_p_eq = {}
# example {"Part1":{"Machine1_Part1_Production":24, ...}}
for p in part_names:
    production_co = {}
    for v in var_list:
        first = v.split("_")[0]
        second = v.split("_")[1]
        if second ==p:
            production_co[v] = production.loc[first, second]
        else:
            production_co[v] = 0
    constraint_left_p_eq[p] = production_co
# constraint_left_p_eq

# setup time for each machine
constraint_left_s_leq = {}
# example {"Machine1":{"Machine1_Part1_Production":8, ...}}
for m in machine_names:
    setup_dic = {}
    for v in var_list:
        first = v.split("_")[0]
        second = v.split("_")[1]
        if first == m:
            setup_dic[v] = setup.loc[first, second]
        else:
            setup_dic[v] = 0
    constraint_left_s_leq[m] = setup_dic
    

# production time for each machine
constraint_left_p_leq = {}
# example {"Machine1":{"Machine1_Part1_Production":24, ...}}
for m in machine_names:
    production_dic = {}
    for v in var_list:
        first = v.split("_")[0]
        second = v.split("_")[1]
        if first == m:
            production_dic[v] = 1
        else:
            production_dic[v] = 0
    constraint_left_p_leq[m] = production_dic
# constraint_left_p_leq

In [8]:
def resolve_model(week):
    # set up constraints name
    # demand
    constraint_right_eq = {}
    for name in part_names:
        constraint_right_eq[name] = demand.loc[week, name]

    m=Model()
    # set up variables
    regular_production_levels = m.addVars(var_list, name="regular")
    overtime_production_levels = m.addVars(var_list, name="overtime")
    setup_levels = m.addVars(var_list, name="setup", vtype = GRB.BINARY)

    # constraint
    eq_constraints_set = m.addConstrs((quicksum(setup_levels[v]*constraint_left_p_eq[p][v]*regular_production_levels[v] for v in var_list) + quicksum(setup_levels[v]*constraint_left_p_eq[p][v]*overtime_production_levels[v] for v in var_list)
                                           == constraint_right_eq[p] for p in part_names ), name="eq_constraints"); 
    
    # regular working hours
    regular_leq = m.addConstrs((quicksum(constraint_left_p_leq[m][vp]*regular_production_levels[vp] for vp in var_list) + quicksum(constraint_left_s_leq[m][vs]*setup_levels[vs] for vs in var_list) 
                                           <= regular_hour_right_leq[m] for m in machine_names), name="regular_leq");
    
    # overtime working hours
    overtime_leq = m.addConstrs((quicksum(constraint_left_p_leq[m][vp]*overtime_production_levels[vp] for vp in var_list)
                                           <= overtime_hour_right_leq[m] for m in machine_names), name="overtime_leq");
    
    # positive constrait
#     regular_positive = m.addConstrs((regular_production_levels[vp] >= 0 for vp in var_list), name="regular_positive");
#     overtime_positive = m.addConstrs((overtime_production_levels[vp] >= 0 for vp in var_list), name="overtime_positive");
    
    # objective function for hours
    obj = quicksum(overtime_production_levels[vp] for vp in var_list)
    m.update()
    # minimize objective
    m.setObjective(obj, GRB.MINIMIZE)
    m.update()
    # optimize model
    m.optimize()
    #reference URL:www.gurobi.com/documentation/8.1/quickstart_mac/py_reporting_results_attri.html
    cost = 0
    regular_hours = 0
    for v in m.getVars():
        if v.x != 0:
            name = v.varName
            n1 = name.split("[")[0]
            n2 = name.split("[")[-1][:-1]
            if n1 == "setup":
                value = setup_co[n2]*v.x
                cost += setup_co[n2]*v.x*100
                regular_hours += setup_co[n2]*v.x
            elif n1 == "regular":
                value = v.x
                cost += 100*v.x
                regular_hours += v.x
            else:
                value = v.x
                cost += 150*v.x
            print('%s %g' % (v.varName, value))  

    print('Obj: %g' % m.ObjVal)
    print('total cost1: %g' % cost)
    print("regular hours: %g" % regular_hours)
    m.reset()

In [9]:
resolve_model(1)

Academic license - for non-commercial use only
Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 1.120894e+02, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  112.08943    0    6          -  112.08943      -     -    0s
H    0     0                     128.0767288  112.08943  12.5%     -    0s
H    0     0                     119.7644578  11

In [10]:
resolve_model(2)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 8.216657e+01, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   82.16657    0    6          -   82.16657      -     -    0s
H    0     0                     102.8144911   82.16657  20.1%     -    0s
H    0     0                      92.3318903   82.16657  11.0%     -    0s
     0     0     cut

In [11]:
resolve_model(3)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [2e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 2.392348e+01, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   23.92348    0    3          -   23.92348      -     -    0s
H    0     0                      32.7057110   23.92348  26.9%     -    0s
H    0     0                      25.7057110   23.92348  6.93%     -    0s
     0     0     cut

In [12]:
resolve_model(4)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 8.860615e+01, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   88.60615    0    6          -   88.60615      -     -    0s
H    0     0                     102.1151904   88.60615  13.2%     -    0s
H    0     0                     100.7818570   88.60615  12.1%     -    0s
*    0     0        

In [13]:
resolve_model(5)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 9.198051e+01, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   91.98051    0    6          -   91.98051      -     -    0s
H    0     0                     107.2433955   91.98051  14.2%     -    0s
H    0     0                      98.9100622   91.98051  7.01%     -    0s
     0     0     cut

In [14]:
resolve_model(6)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [2e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 6.901568e+01, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   69.01568    0    5          -   69.01568      -     -    0s
H    0     0                      83.6036464   69.01568  17.4%     -    0s
H    0     0                      75.8677156   69.01568  9.03%     -    0s
     0     0   72.60

In [15]:
resolve_model(7)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [2e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 6.873712e+01, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   68.73712    0    3          -   68.73712      -     -    0s
H    0     0                      76.3985181   68.73712  10.0%     -    0s
     0     0     cutoff    0        76.39852   76.39852  0.00%     -    0s

Cutting planes:
  G

In [16]:
resolve_model(8)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [2e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 1.114552e+02, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  111.45518    0    5          -  111.45518      -     -    0s
H    0     0                     129.7806915  111.45518  14.1%     -    0s
H    0     0                     122.7806915  111.45518  9.22%     -    0s
     0     0     cut

In [17]:
resolve_model(9)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 1.267085e+01, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   12.67085    0    6          -   12.67085      -     -    0s
H    0     0                      48.6891608   12.67085  74.0%     -    0s
     0     0   24.82422    0    8   48.68916   24.82422  49.0%     -    0s
     0     0   25.82

In [18]:
resolve_model(10)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 0.000000e+00, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6          -    0.00000      -     -    0s
H    0     0                      23.9088689    0.00000   100%     -    0s
     0     0    5.45800    0    5   23.90887    5.45800  77.2%     -    0s
     0     0    6.27

In [19]:
resolve_model(11)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [4e+03, 5e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.00s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Explored 1 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: b"Unable to retrieve attribute 'x'"

In [20]:
resolve_model(12)

Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 45 columns
Presolve time: 0.01s
Presolved: 75 rows, 50 columns, 190 nonzeros
Variable types: 40 continuous, 10 integer (10 binary)

Root relaxation: objective 8.216657e+01, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   82.16657    0    6          -   82.16657      -     -    0s
H    0     0                     102.8144911   82.16657  20.1%     -    0s
H    0     0                      92.3318903   82.16657  11.0%     -    0s
     0     0     cut